<a href="https://colab.research.google.com/github/suaJeong-777/AI-recommend-system/blob/main/eco_friendly_product_recommendation0816.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import warnings; warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

movies = pd.read_csv(r'/content/drive/MyDrive/소개딩_2022/tmdb_5000_environment.csv',encoding='cp949')
# print(movies.shape)
# print(movies.head(10))

movies_df = movies[['id','title','genres','voteAverage', 'voteCount', 'popularity', 'keywords', 'overview']]
print(movies_df['genres'].head(1))
print(movies_df['keywords'].head(1))

0    [{"id": 6, "name": "clothes"}, {"id": 63, "name": "bag"}, {"id": 10, "name": "material"}, {"id": 103, "name": "plastic"}]
Name: genres, dtype: object
0    [{"id": 63, "name": "bag"}, {"id": 103, "name": "plastic"}]
Name: keywords, dtype: object


In [4]:
from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
print(movies_df['genres'].head(1))
print(movies_df['keywords'].head(1))

0    [clothes, bag, material, plastic]
Name: genres, dtype: object
0    [bag, plastic]
Name: keywords, dtype: object


In [5]:
from sklearn.feature_extraction.text import CountVectorizer

#CountVectorizer를 적용하기 위해 공백문자로 word단위가 구분되는 문자열로 변환.
movies_df['generes_literal'] = movies_df['genres'].apply(lambda x :  (' ').join(x))
print(movies_df['generes_literal'].head(10))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['generes_literal'])
print(genre_mat)
print(genre_mat.shape)

0         clothes bag material plastic
1         clothes pouch matrial fabric
2         clothes bag material leather
3          clothes bag material others
4          clothes top material fabric
5    clothes underwear material others
6      clothes wallet material leather
7     beauty toothbrush material woods
8       kitchen others material others
9    kitchen dishcloth material fabric
Name: generes_literal, dtype: object
  (0, 33)	1
  (0, 25)	1
  (0, 107)	1
  (0, 126)	1
  (0, 34)	1
  (0, 26)	1
  (0, 113)	1
  (1, 33)	1
  (1, 127)	1
  (1, 115)	1
  (1, 67)	1
  (1, 35)	1
  (1, 128)	1
  (1, 116)	1
  (2, 33)	1
  (2, 25)	1
  (2, 107)	1
  (2, 34)	1
  (2, 26)	1
  (2, 104)	1
  (2, 111)	1
  (3, 33)	1
  (3, 25)	1
  (3, 107)	1
  (3, 34)	1
  :	:
  (95, 138)	1
  (95, 130)	1
  (95, 83)	1
  (95, 9)	1
  (95, 139)	1
  (95, 131)	1
  (95, 117)	1
  (95, 86)	1
  (95, 118)	1
  (96, 10)	1
  (96, 0)	1
  (96, 130)	1
  (96, 83)	1
  (96, 131)	1
  (96, 58)	1
  (96, 92)	1
  (96, 63)	1
  (96, 93)	1
  (96, 1)	1
  (9

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:1])

(98, 98)
[[1.         0.14285714 0.71428571 0.71428571 0.28571429 0.28571429
  0.28571429 0.14285714 0.12598816 0.14285714 0.14285714 0.14285714
  0.14285714 0.14285714 0.12598816 0.28571429 0.14285714 0.14285714
  0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
  0.12598816 0.71428571 0.71428571 0.71428571 0.14285714 0.14285714
  0.12598816 0.14285714 0.14285714 0.14285714 0.12598816 0.14285714
  0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.      

In [7]:
genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_ind[:1])

[[ 0  3 26 27 25  2  4  6 15  5 31 40 28 29 33 32 35 22 37 38 39 36 23 21
  13  1  7  9 10 12 11 20 16 17 18 19  8 24 14 30 34 96 41 42 43 44 45 46
  47 97 49 50 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94
  95 74 73 72 60 51 52 53 54 55 56 57 58 59 61 71 62 63 64 65 66 67 68 69
  70 48]]


In [29]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    #인자로 입력된 movies_df DataFrame에서 'title'칼럼이 입력된 title_name값인 DataFrame 추출
    title_movie = df[df['title'] == title_name]

    #title_named을 가진 DataFrame의 index객체를 ndarray로 반환하고
    #sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]

    #추출된 top_n index출력, top_n index는 2차원 데이터임
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)

    return df.iloc[similar_indexes]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, '대나무 슬리퍼', 10)   # 찾고자 하는 데이터와 유사한 것
print(similar_movies.info())
print(similar_movies)
print(similar_movies[['title', 'voteCount', 'voteAverage']])

[[15 36 13 40 32 33  7 39 38 37]]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 15 to 37
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               10 non-null     int64  
 1   title            10 non-null     object 
 2   genres           10 non-null     object 
 3   voteAverage      10 non-null     float64
 4   voteCount        10 non-null     int64  
 5   popularity       10 non-null     float64
 6   keywords         10 non-null     object 
 7   overview         10 non-null     object 
 8   generes_literal  10 non-null     object 
 9   weighted_vote    10 non-null     float64
dtypes: float64(3), int64(2), object(5)
memory usage: 880.0+ bytes
None
        id          title                                   genres  \
15    2454        대나무 슬리퍼        [clothes, shoes, material, woods]   
36   91314  wasara 원형접시 S       [kitchen, dishes, material, woods]   
13   57201          소나무볼펜  [

In [30]:
print(movies_df[['title', 'voteCount', 'voteAverage']].sort_values('voteAverage', ascending=False)[:10])

                      title  voteCount  voteAverage
64          The Dark Knight      12002          8.2
95                Inception      13752          8.1
94             Interstellar      10867          8.1
76               Inside Out       6560          8.0
93  Guardians of the Galaxy       9742          7.9
57                   WALL쨌E       6296          7.8
87               Big Hero 6       6135          7.8
65                       Up       6870          7.7
42              Toy Story 3       4597          7.6
22                텀블러 드링크 백       4524          7.6


In [31]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   genres       98 non-null     object 
 1   id           98 non-null     int64  
 2   keywords     98 non-null     object 
 3   overview     98 non-null     object 
 4   popularity   98 non-null     float64
 5   title        98 non-null     object 
 6   voteAverage  98 non-null     float64
 7   voteCount    98 non-null     int64  
 8   imageSource  41 non-null     object 
dtypes: float64(2), int64(2), object(5)
memory usage: 7.0+ KB


In [32]:
#movies=movies.astype({'voteCount':'float64'})

In [33]:
percentile = 0.6
m = movies['voteCount'].quantile(percentile)
C = movies['voteAverage'].mean()

def weighted_vote_average(record):
    v = record['voteCount']
    R = record['voteAverage']

    return ( (v/(v+m)) * R) + ( (m/(m+v))*C )

movies_df['weighted_vote'] = movies.apply(weighted_vote_average, axis=1)
movies_df[['title', 'weighted_vote', 'voteAverage', 'voteCount']].sort_values('weighted_vote', ascending=False)[:10]
print(movies_df['title'].head(100))

def find_sim_movie_weight(df, sorted_ind, title_name, top_n=10):
    #인자로 입력된 movies_df DataFrame에서 'title'칼럼이 입력된 title_name값인 DataFrame 추출
    title_movie = df[df['title'] == title_name]
    print(title_movie)
    title_index = title_movie.index.values

    #top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)

    #기준 영화 인덱스는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]


    #top_n의 2배에 해당하는 후보군에서 weight_vote가 노은 순으로 top_n만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie_weight(movies_df, genre_sim_sorted_ind, 'Inception', 10)
print(similar_movies[['title', 'voteAverage', 'weighted_vote']])

0                                                             플리츠 백팩
1                                                              심플파우치
2                                                            프린트 클러치
3                                                                 백팩
4                                                                오브티
5                                                                수영복
6                                                               명함지갑
7                                                              대나무칫솔
8                                                              설거지비누
9                                                               삼베행주
10                                                             소창손수건
11                                                   제로웨이스트 셀룰로오스 행주
12                                                              드링크백
13                                                             소나무볼펜
14                                

In [34]:
print(similar_movies[['title', 'voteAverage']])

                                  title  voteAverage
93              Guardians of the Galaxy          7.9
84  Captain America: The Winter Soldier          7.6
67                             Iron Man          7.4
73                     Edge of Tomorrow          7.6
46           X-Men: Days of Future Past          7.5
47              Star Trek Into Darkness          7.4
51                          Pacific Rim          6.7
78                           Iron Man 2          6.6
56                     Star Trek Beyond          6.6
82                           The Lovers          4.8
